In [ ]:
import importlib
from importlib import reload
import ado_commits
import github_commits
import gen_note
reload(ado_commits)
reload(github_commits)
reload(gen_note)

In [ ]:
import argparse
import json
from concurrent.futures import ThreadPoolExecutor
from datetime import date, datetime
from typing import Any

from ado_commits import get_commits_info_for_ids as get_commits_info_for_ids
from github_commits import get_commit_info as get_github_commit_info
from gen_note import generate_release_note

def _sanitize_for_json(obj: Any) -> Any:
	if isinstance(obj, (datetime, date)):
		return obj.isoformat()
	if isinstance(obj, dict):
		return {key: _sanitize_for_json(value) for key, value in obj.items()}
	if isinstance(obj, (list, tuple, set)):
		return [_sanitize_for_json(item) for item in obj]
	if hasattr(obj, "__dict__"):
		public_attrs = {
			key: value
			for key, value in obj.__dict__.items()
			if not key.startswith("_")
		}
		if public_attrs:
			return _sanitize_for_json(public_attrs)
	return obj

parser = argparse.ArgumentParser(
    description="Fetch Azure DevOps commit + PR data between two commits (inclusive).",
)



In [ ]:
ado_commits = []
github_commits = []

# generate ado.txt & githubt.txt from e.g. `git v1.0.0..master --format="%H" > github.txt`
with open('in/ado.txt') as ado_commit_file:
    ado_commits += ado_commit_file.read().splitlines()

with open('in/github.txt') as github_commit_file:
    github_commits += github_commit_file.read().splitlines()

print(len(ado_commits))
print(len(github_commits))

In [ ]:
ado_commit_infos = get_commits_info_for_ids(ado_commits[:2], max_parallel=12)
github_commit_infos = get_github_commit_info(github_commits[:2], max_parallel=12)
commit_infos = ado_commit_infos + github_commit_infos
payloads = [json.dumps(_sanitize_for_json(info), ensure_ascii=False, indent=2) for info in commit_infos]
with open("out/release_notes_1.1_payloads.json", "w", encoding="utf-8") as f:
	f.write("[\n")
	f.write(",\n".join(payloads))
	f.write("\n]\n")

In [ ]:
notes = []
with ThreadPoolExecutor(max_workers=12 or len(payloads) or 1) as pool:
    release_notes = list(pool.map(generate_release_note, payloads))
for release_note in release_notes:
    notes.append(json.dumps(release_note, ensure_ascii=False))
notes


In [ ]:
dict_notes = [json.loads(note) for note in notes]
sorted_notes = sorted(dict_notes, key=lambda x: datetime.fromisoformat(x['merged_at']), reverse=True)

In [ ]:
sorted_notes

In [ ]:

working_notes = sorted_notes.copy()
for note in working_notes:
    PR_LINK = ""
    if note['pr_number'] is not None:
        if "Internal" in note['repo_name']:
            PR_LINK = f"https://msdata.visualstudio.com/A365/_git/SynapseML-Internal/pullrequest/{note['pr_number']}"
        else:
            PR_LINK = f"https://github.com/microsoft/SynapseML/pull/{note['pr_number']}"
    print(note['ONELINE'])
    if len(note['ONELINE'].split(' - ')) > 1:
        ONELINE = f"${note['ONELINE'].split(' - ')[1]}"
    ONELINE = ONELINE.split("[#")[0].strip()
    ONELINE = f"{ONELINE} [#{note['pr_number']}]({PR_LINK})"
    note['ONELINE'] = ONELINE
    print(note['ONELINE'])
    print(note['pr_title'])
    print()

with open("out/release_notes_1.1_commits.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(sorted_notes, ensure_ascii=False, indent=2))

with open("out/release_notes_1.1_one_liners.txt", "w", encoding="utf-8") as f:
    f.writelines([note['ONELINE'] for note in working_notes])